In [22]:
import pandas as pd
import re

In [23]:
# load up transposed tunes
df = pd.read_csv('data/transposed_tunes.csv')

In [24]:
df.head()

,Unnamed: 0.1,Unnamed: 0,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc
0,0,0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:..."
1,1,1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...
2,2,2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...
3,3,3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...
4,4,4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,..."


In [25]:
df.transposed_abc[34]

'X:9294\nT:18-6 Two Step\nM:6/8\nK:Cmajor\nA^GA dcA|G^FG c3|DEF E2 D|C3 z2G|\n|:G^FG AGF|GAG E3|G^FG AGE|F3 z3|\nG,^G,A, B,=B,C|D2 E F2 ^F|G^FG AGF|E3 z3|\nG^FG AGF|GAG E3|G^FG c2 B|A3 z3|\nA^GA dcA|G^FG c3|DEF E2 D|1 C3 z2 G:|2 C3 z2 D||\n|:G2 D A2 D|B3 G2 ^F|E^DE G2 E|D3 z2 D|\nG2 D A2 D|B3 G2 B|A2 E ^F2 G|A3 z2 D|\nG2 D A2 D|B3 G2 ^F|E2 ^G A2 B|c3 z3|\nE^DE G^FE|DGB d2 c|BAG D2 ^F|1 G3 z2 D:|2 G3 z2 G||\nG^FG AGF|GAG E3|G^FG AGE|F3 z3|\nG,^G,A, B,=B,C|D2 E F2 ^F|G^FG AGF|E3 z3|\nG^FG AGF|GAG E3|GAG c2 B|A3 z3|\nA^GA dcA|G^FG c3|DEF E2 D|C3 z2 B,||\nA,3 ^G,2 A,|F,6|A,^G,A, DCA,|B,6|\nG,3 ^F,2 G,|E,6|C,D,E, F,G,^G,|A,6|\nA,3 ^G,2 A,|F,6|FEF G2 F|D6|\nD^CD FED|C=B,C F3|G,A,B, A,2 G,|F,3 G3||'

In [45]:
# abc_tokenizer.py
# Converts ABC notation into note-duration tokens (note_seq + dur_seq) in quarter-note units

import re
from typing import Tuple, List

# Regex for header lines and event tokens (broken-rhythm markers and notes/rests)
HEADER_RE = re.compile(r'^(X|T|M|L|K):\s*(.+)$', re.MULTILINE)
EVENT_RE = re.compile(r'([<>]+)|([=^_]*[A-Ga-gzZxZ])([0-9/]*)')

# Fallback unit length based on meter's denominator
METER_BASE_UNIT = {
    2: 1/8,
    3: 1/8,
    4: 1/8,
    5: 1/8,
    6: 1/8,
    8: 1/8,
    9: 1/8,
    12: 1/8,
    16: 1/16
}


def extract_header(abc_str: str) -> dict:
    """Extract ABC header fields X,T,M,L,K into a dict."""
    header = {}
    for m in HEADER_RE.finditer(abc_str):
        field, value = m.groups()
        header[field] = value.strip()
    return header


def infer_unit_length(header: dict) -> float:
    """Return the unit note length (L:) or fallback by meter denominator."""
    # Try explicit L:
    if 'L' in header:
        try:
            return eval(header['L'])
        except Exception:
            pass
    # Fallback to meter denominator
    meter = header.get('M', '4/4')
    try:
        denom = int(meter.split('/')[-1])
        return METER_BASE_UNIT.get(denom, 1/8)
    except Exception:
        return 1/8


def parse_meter_quarters(meter: str) -> float:
    """Convert meter M: to whole-note fraction in quarters (e.g. 3/4->3 quarters)."""
    try:
        num, den = meter.split('/')
        num_f, den_f = float(num), float(den)
        return (num_f / den_f) * 4.0
    except Exception:
        return 4.0


def abc_length_to_float(length: str) -> float:
    """Parse ABC length notation (e.g. '3/2','/','2') into a multiplier."""
    if not length:
        return 1.0
    if '/' in length:
        if length == '/':
            return 0.5
        parts = length.split('/')
        if len(parts) == 2:
            num, den = parts
            # exact fraction
            if num and den:
                try:
                    return float(num) / float(den)
                except Exception:
                    return 1.0
            # shorthand numerator only
            if num:
                try:
                    return float(num) / 2.0
                except Exception:
                    return 1.0
            # shorthand denominator only
            if den:
                try:
                    return 1.0 / float(den)
                except Exception:
                    return 1.0
        return 1.0
    else:
        try:
            return float(length)
        except Exception:
            return 1.0


def tokenize_abc_note_and_dur(abc_str: str) -> Tuple[List[str], List[float]]:
    """
    Parse an ABC tune into parallel note_seq and dur_seq lists.
    Supports broken rhythms ('<'/'>') and multi-measure rests 'Z'.
    Durations output in quarter-note units.
    """
    header = extract_header(abc_str)
    unit = infer_unit_length(header)      # length of one unit in whole-notes
    measure_quarters = parse_meter_quarters(header.get('M', '4/4'))

    # Remove header lines
    lines = [ln for ln in abc_str.split('\n') if not HEADER_RE.match(ln)]
    text = ' '.join(lines)

    # Parse events: either broken-rhythm markers or note/rest tokens
    events = []
    for m in EVENT_RE.finditer(text):
        if m.group(1):  # broken rhythm marker
            events.append({'type': 'mark', 'val': m.group(1)})
        else:
            pitch = m.group(2)
            length_str = m.group(3)
            events.append({'type': 'note', 'pitch': pitch, 'length': length_str})

    note_seq: List[str] = []
    dur_seq: List[float] = []
    pending_mark = None
    prev_idx = None

    for evt in events:
        if evt['type'] == 'mark':
            pending_mark = evt['val']
            continue

        pitch = evt['pitch']
        length_str = evt['length']
        base_mult = abc_length_to_float(length_str)

        # Multi-measure rest
        if pitch.upper() == 'Z' and length_str.isdigit():
            dur = int(length_str) * measure_quarters
        else:
            # convert ABC-unit to quarter notes: unit is fraction of whole-note
            dur = base_mult * (unit * 4.0)

        # Apply broken rhythm if present
        if pending_mark:
            k = len(pending_mark)
            if pending_mark[0] == '>':
                prev_mult = 1 + sum(1/(2**i) for i in range(1, k+1))
                next_mult = 1/(2**k)
            else:  # '<'
                prev_mult = 1/(2**k)
                next_mult = 1 + sum(1/(2**i) for i in range(1, k+1))

            # adjust previous duration
            if prev_idx is not None:
                dur_seq[prev_idx] *= prev_mult
            # adjust current duration
            dur *= next_mult
            pending_mark = None

        # record normalized pitch and duration
        normalized_pitch = pitch.replace('Z', 'z').replace('x', 'z')
        note_seq.append(normalized_pitch)
        dur_seq.append(dur)
        prev_idx = len(dur_seq) - 1

    return note_seq, dur_seq


def tokenize_abc_notes(abc_str: str) -> List[str]:
    """Backward compatible: combined token list 'note_duration'."""
    notes, durs = tokenize_abc_note_and_dur(abc_str)
    return [f"{n}_{d:.3f}" for n, d in zip(notes, durs)]


def generate_ngrams(tokens: List[str], n: int = 3) -> List[str]:
    """Generate sliding n-grams of size n from token list."""
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


In [46]:
# Keep only rows where transposed_abc is a string
df = df[df['transposed_abc'].apply(lambda x: isinstance(x, str))]
df[['note_seq','dur_seq']] = df['transposed_abc'].apply(
    lambda s: pd.Series(tokenize_abc_note_and_dur(s))
)

In [49]:
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc,note_seq,dur_seq
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ...","[G, c, B, A, G, E, D, C, A, B, C, A, B, C, E, ...","[0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5..."
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ...","[G, E, E, E, D, E, D, C, G, E, E, E, F, G, A, ...","[0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ...","[F, D, D, D, D, C, D, C, B, F, D, D, D, E, F, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ...","[C, D, E, C, F, D, E, C, A, A, D, C, B, G, G, ...","[0.75, 0.25, 0.75, 0.25, 0.75, 0.25, 0.75, 0.2..."
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ...","[G, B, C, C, E, G, F, D, E, C, G, A, D, C, B, ...","[0.5, 0.5, 0.5, 1.0, 0.75, 0.25, 0.75, 0.25, 0..."


In [47]:
print(df.transposed_abc[0])

X:35234
T:$150 Boot, The
M:2/4
K:Cmajor
|:G>c BA|GE DC|A,/B,/C A,/B,/C|ED CB,|
G>c BA|GE DC|A,/B,/C A,/B,/C|ED C2:|
|:A>B cd|ed cB|A/B/c A/B/c|G/F/E/D/ ED|
A>B cd|ed cB|A/B/c A/B/c|dc c2:|


In [48]:
print(df.dur_seq[0])

[0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.5, 0.25, 0.25, 0.5, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.5, 0.25, 0.25, 0.5, 0.5, 0.5, 1.0]


In [32]:
df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [50]:
df.to_pickle('data/tokenized_dataset.pkl')